In [11]:
import torch
import torch.optim as optim
from utils import Dataset, load_model, save_model
from training import Scheduler, train, predict, prompt
from datasets import load_dataset
from tokenizer import Nerdimizer
from transformer import Transformer

device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
datadict = load_dataset("europa_eac_tm", language_pair=("en", "de"), split="train")

Using the latest cached version of the module from /Users/tonimo/.cache/huggingface/modules/datasets_modules/datasets/europa_eac_tm/955b2501a836c2ea49cfe3e719aec65dcbbc3356bbbe53cf46f08406eb77386a (last modified on Tue Nov 22 19:20:52 2022) since it couldn't be found locally at europa_eac_tm., or remotely on the Hugging Face Hub.
Using custom data configuration en2de-176cd02372067e72
Found cached dataset europa_eac_tm (/Users/tonimo/.cache/huggingface/datasets/europa_eac_tm/en2de-176cd02372067e72/0.0.0/955b2501a836c2ea49cfe3e719aec65dcbbc3356bbbe53cf46f08406eb77386a)


In [13]:
inputs = [pair["en"] for pair in datadict["translation"]]
labels = [pair["de"] for pair in datadict["translation"]]
dataset = Dataset(inputs, labels)

In [14]:
df = dataset.dataframe()
df.head(5)

,inputs,labels
0,Nr. teachers/trainers,Anzahl Lehrer(innen)/Trainer
1,APPLICANT,ANTRAGSTELLERIN/ANTRAGSTELLER
2,The grant application will be processed by com...,Der Förderantrag wird elektronisch verarbeitet...
3,To be signed by the person legally authorised ...,"Unterschrift der Person, die rechtsverbindlich..."
4,DATE OF BIRTH,GEBURTSDATUM


In [15]:
print(df.isnull().values.any())
df.describe()

False


,inputs,labels
count,4473,4473
unique,4452,4312
top,Event,x
freq,2,29


In [16]:
dataset.sample()

('Improve learners comprehension and expression in a language',
 'Verbesserung des Verständnisses und des Ausdrucks von Lernenden in einer Sprache')

In [17]:
corpus = dataset.corpus()
tokenizer = Nerdimizer()
tokenizer.train(corpus)
vocab_size = len(tokenizer)
maxlen = dataset.avg_tokenized_len(tokenizer) * 2
start, end, pad = tokenizer["[S]"], tokenizer["[E]"], tokenizer["[P]"]
tokenizer.padon(maxlen, pad_id=pad)
tokenizer.truncon(maxlen)
print(f"Number of word piece tokens: {len(tokenizer)}")

Number of word piece tokens: 15693


In [18]:
tokenized_dataset = dataset.tokenized(tokenizer)
dataloader = tokenized_dataset.dataloader(batch_size=32, drop_last=False)

In [19]:
model = Transformer(vocab_size=len(tokenizer), maxlen=maxlen, pad_id=pad, 
                    dm=512, nhead=8, layers=4, dff=1024)
optimizer = optim.Adam(model.parameters(), lr=1e-5, betas=(0.9, 0.98), eps=10e-9)
scheduler = Scheduler(optimizer, dm=512, warmup=100)

In [21]:
train(model, optimizer, scheduler, dataloader, epochs=2500, verbose=True, device=device)

Training Started
Epoch 1 Started


KeyboardInterrupt: 